In [1]:
import numpy as np
from pydtk2 import *
from pymoab.core import Core
from pymoab.types import MBQUAD, MB_TYPE_DOUBLE, MB_TAG_DENSE
from mpi4py import MPI

In [2]:
# use our modified meshio
from meshio_modified.vtk_io import read
from meshio_modified.vtu_io import write

In [3]:
coarse_file = 'meshes/CubSph2_0_0_0_41_55_63_GaussHillSlotCylnd.vtk'
fine_file = 'meshes/CubSph2_0_0_0_73_81_101_GaussHillSlotCylnd.vtk'

In [4]:
# the next line should be modified for maximum iterations
itmax = 20
# the next line should be modified for output file names
output_common = 'results/DTKresults'
output_fine, output_coarse = output_common + '_fine', output_common + '_coarse'
# the next line indicates the Radial Basis Function Radius
radius = 0.3

In [5]:
# read data in
coors_fine, elem_fine, point_data_fine, _, _ = read(fine_file)
coors_coarse, elem_coarse, point_data_coarse, _, _ = read(coarse_file)

In [6]:
# elem_? and point_data_? stored in dictionaries, we make references to
# the values for them
conn_fine = elem_fine['quad']
conn_coarse = elem_coarse['quad']
field_values_fine = point_data_fine['scalars']
field_values_coarse = point_data_coarse['scalars']

In [7]:
# MOAB stuffs

mdb_fine, mdb_coarse = Core(), Core()

In [8]:
# Read in vertices, get the return Range instances, for later use
rng_fine = mdb_fine.create_vertices(coors_fine)
rng_coarse = mdb_coarse.create_vertices(coors_coarse)

In [9]:
# NOTE that pyMOAB takes 1 based connectivity table, and dtype must be uint64!

conn_fine_moab = np.array(conn_fine, dtype=np.uint64) + rng_fine[0]
conn_coarse_moab = np.array(conn_coarse, dtype=np.uint64) + rng_coarse[0]

In [10]:
# create Q1 elements
rng_quad_fine = mdb_fine.create_elements(MBQUAD, conn_fine_moab)
rng_quad_coarse = mdb_coarse.create_elements(MBQUAD, conn_coarse_moab)
# create adj for the elements associated to vertices
mdb_fine.get_adjacencies(rng_quad_fine, to_dim=0, create_if_missing=True)
mdb_fine.get_adjacencies(rng_quad_coarse, to_dim=0, create_if_missing=True)

In [11]:
# now we create fields
field_fine, field_coarse = 'temperatureFine', 'temperatureCoarse'
# create tags
tag_fine = mdb_fine.tag_get_handle(
    field_fine,
    size=1,
    tag_type=MB_TYPE_DOUBLE,
    storage_type=MB_TAG_DENSE,
    create_if_missing=True)
tag_coarse = mdb_coarse.tag_get_handle(
    field_coarse,
    size=1,
    tag_type=MB_TYPE_DOUBLE,
    storage_type=MB_TAG_DENSE,
    create_if_missing=True)

In [12]:
# put data values into meshdbs
mdb_fine.tag_set_data(tag_fine, rng_fine, field_values_fine)
mdb_coarse.tag_set_data(tag_coarse, rng_coarse, field_values_coarse)

In [13]:
# Now construct two PyDTK2 MOAB managers
fine_manager = DTK2MoabManager(mdb_fine)
coarse_manager = DTK2MoabManager(mdb_coarse)

In [14]:
# now we push the tag strings into the managers
fine_manager.register_tag(field_fine)
coarse_manager.register_tag(field_coarse)

In [15]:
# next step is to construct the DTK2 MOAB mappers
fine2coarse = DTK2Mapper(source=fine_manager, target=coarse_manager, r=radius)
coarse2fine = DTK2Mapper(source=coarse_manager, target=fine_manager, r=radius)

# now register coupled flags
fine2coarse.register_coupled_tags(field_fine, field_coarse)
coarse2fine.register_coupled_tags(field_coarse, field_fine)

In [17]:
buffer1 = np.empty(field_values_fine.size, dtype=np.float64)
buffer2 = np.empty(field_values_coarse.size, dtype=np.float64)
for it in range(itmax):
    # retrieve the values
    buffer2[:] = mdb_coarse.tag_get_data(tag_coarse, rng_coarse).reshape(-1)
    # dump to file
    write(output_coarse + '%i.vtu' % it,
          coors_coarse,
          elem_coarse,
          write_binary=False,
          point_data={field_coarse: buffer2})
    # transfer from fine to coarse
    fine2coarse.apply(field_fine, field_coarse)
    
    # retrive the values
    buffer1[:] = mdb_fine.tag_get_data(tag_fine, rng_fine).reshape(-1)
    # dump to file
    write(output_fine + '%i.vtu' % it,
          coors_fine,
          elem_fine,
          write_binary=False,
          point_data={field_fine: buffer1})
    # transfer from coarse to fine
    coarse2fine.apply(field_coarse, field_fine)

In [18]:
# write the final solutions
# retrieve the values
buffer2[:] = mdb_coarse.tag_get_data(tag_coarse, rng_coarse).reshape(-1)
    # dump to file
write(output_coarse + '%i.vtu' % itmax,
      coors_coarse,
      elem_coarse,
      write_binary=False,
      point_data={field_coarse: buffer2})
    
# retrive the values
buffer1[:] = mdb_fine.tag_get_data(tag_fine, rng_fine).reshape(-1)
# dump to file
write(output_fine + '%i.vtu' % itmax,
      coors_fine,
      elem_fine,
      write_binary=False,
      point_data={field_fine: buffer1})